Question 3: Clustering the Toronto Neighbourhoods.

In [29]:
# Import the necessary libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Solving environment: ...working... done

# All requested packages already installed.



In [30]:
df = pd.read_csv('australian_postcodes.csv')
df_mel = df[(df['state']=='VIC') & (df['type']=='Delivery Area')]
df_mel.head(20)

,id,postcode,locality,state,long,lat,dc,type,status,sa3,sa3name,sa4,sa4name
6099,4746,3000,MELBOURNE,VIC,144.956776,-37.817403,CITY DELIVERY CENTRE,Delivery Area,Updated 6-Feb-2020,20604.0,Melbourne City,206.0,Melbourne - Inner
6101,4748,3002,EAST MELBOURNE,VIC,144.982207,-37.818517,CITY DELIVERY CENTRE,Delivery Area,Updated 6-Feb-2020,20604.0,Melbourne City,206.0,Melbourne - Inner
6102,4749,3003,WEST MELBOURNE,VIC,144.949592,-37.810871,CITY DELIVERY CENTRE,Delivery Area,Updated 6-Feb-2020,20604.0,Melbourne City,206.0,Melbourne - Inner
6103,4750,3004,MELBOURNE,VIC,144.970161,-37.844246,CITY DELIVERY CENTRE,Delivery Area,Updated 6-Feb-2020,20605.0,Port Phillip,206.0,Melbourne - Inner
6106,4752,3005,WORLD TRADE CENTRE,VIC,144.950858,-37.824608,CITY DELIVERY CENTRE,Delivery Area,Updated 6-Feb-2020,20604.0,Melbourne City,206.0,Melbourne - Inner
6107,20754,3006,SOUTH WHARF,VIC,144.952074,-37.825287,Melbourne Metro,Delivery Area,Updated 6-Feb-2020,20605.0,Port Phillip,206.0,Melbourne - Inner
6108,4753,3006,SOUTHBANK,VIC,144.965926,-37.823258,SOUTH MELBOURNE DC,Delivery Area,Updated 6-Feb-2020,20605.0,Port Phillip,206.0,Melbourne - Inner
6109,4754,3008,DOCKLANDS,VIC,144.948039,-37.814719,CITY DELIVERY CENTRE,Delivery Area,Updated 6-Feb-2020,20604.0,Melbourne City,206.0,Melbourne - Inner
6110,4755,3010,UNIVERSITY OF MELBOURNE,VIC,144.961351,-37.796152,CARLTON DC,Delivery Area,Updated 6-Feb-2020,20604.0,Melbourne City,206.0,Melbourne - Inner
6111,4756,3011,FOOTSCRAY,VIC,144.907953,-37.807101,WEST FOOTSCRAY DC,Delivery Area,Updated 6-Feb-2020,21303.0,Maribyrnong,213.0,Melbourne - West


In [31]:
df_mel = df_mel.dropna()
df_mel.rename(columns={'sa3name':'neighbourhood', 'sa4name':'region'},inplace=True)
df_mel.drop(columns=['id','postcode','dc','locality','type','sa3','status','sa4'],inplace=True)
df_mel

new_df = df_mel.groupby(['neighbourhood']).count()
new_df.drop(columns=['long','lat'],inplace=True)
new_df.rename(columns={'region':'count'},inplace=True)
new_df
  
# Observe the result 
df_mel = df_mel[['long','lat','neighbourhood']]
df_mel = df_mel.drop_duplicates(subset = ['neighbourhood'])
df_mel.reset_index(drop=True,inplace=True)
df_mel

,long,lat,neighbourhood
0,144.956776,-37.817403,Melbourne City
1,144.970161,-37.844246,Port Phillip
2,144.907953,-37.807101,Maribyrnong
3,144.880556,-37.838242,Hobsons Bay
4,144.829945,-37.777204,Brimbank
5,144.770641,-37.782720,Melton - Bacchus Marsh
6,144.649936,-37.750456,Wyndham
7,144.861940,-37.761235,Keilor
8,144.922995,-37.766184,Essendon
9,144.901104,-37.726746,Moreland - North


In [32]:
# plot map of Melbourne and its neighbourhoods

latitude = -37.8136
longitude = 144.9631

map_mel = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighbourhood in zip(df_mel['lat'], df_mel['long'], df_mel['neighbourhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mel)  

map_mel

# Step 1: Send a GET request to the FourSquare API for venue data

In [33]:
# FourSquare Login
CLIENT_ID = '2TVOB50BRP5EIPTVZQ2NXGMGSEEYBAWLDFCF4NDL5V2G40IE' # your Foursquare ID
CLIENT_SECRET = 'N1CVKFU13VDXGMWFVXZ1S4JTMQD1BKOXABHWFKUYCEYRRG2U' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2TVOB50BRP5EIPTVZQ2NXGMGSEEYBAWLDFCF4NDL5V2G40IE
CLIENT_SECRET:N1CVKFU13VDXGMWFVXZ1S4JTMQD1BKOXABHWFKUYCEYRRG2U


In [34]:
df_mel.loc[0, 'neighbourhood']

'Melbourne City'

In [35]:
radius = 5000
LIMIT = 100
latitude = -37.8136
longitude = 144.9631
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=2TVOB50BRP5EIPTVZQ2NXGMGSEEYBAWLDFCF4NDL5V2G40IE&client_secret=N1CVKFU13VDXGMWFVXZ1S4JTMQD1BKOXABHWFKUYCEYRRG2U&ll=-37.8136,144.9631&v=20180605&radius=5000&limit=100'

In [36]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ec0d7e5d03993001b492297'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Melbourne',
  'headerFullLocation': 'Melbourne',
  'headerLocationGranularity': 'city',
  'totalResults': 243,
  'suggestedBounds': {'ne': {'lat': -37.76859995499996,
    'lng': 145.01995504330822},
   'sw': {'lat': -37.858600045000045, 'lng': 144.90624495669178}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54631f6e498ed0dde017e53c',
       'name': 'Tipo 00',
       'location': {'address': '361 Little Bourke St',
        'lat': -37.81352651659617,
        'lng': 144.96197769842925,
        'labeledLatLngs': [{'label': 'display',
          'lat': -37.81352651659617,


In [37]:
# Define a "get venue categories" function
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name'] 
    
# load and structure the venues into a dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\Ervin\anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Tipo 00,Italian Restaurant,-37.813527,144.961978
1,Little Rogue Coffee,Coffee Shop,-37.810911,144.964027
2,Boilermaker House,Whisky Bar,-37.811320,144.966155
3,Shortstop Coffee & Donuts,Donut Shop,-37.811240,144.961000
4,Brother Baba Budan,Coffee Shop,-37.813445,144.962137


In [38]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


# Step 2: Explore Call for all Toronto neighbourhoods

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [40]:
# Create a new dataframe called Melb neighbourhoods

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

melbourne_venues = getNearbyVenues(names=df_mel['neighbourhood'],
                                   latitudes=df_mel['lat'],
                                   longitudes=df_mel['long']
                                  )

C:\Users\Ervin\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


Melbourne City
Port Phillip
Maribyrnong
Hobsons Bay
Brimbank
Melton - Bacchus Marsh
Wyndham
Keilor
Essendon
Moreland - North
Tullamarine - Broadmeadows
Yarra
Brunswick - Coburg
Darebin - South
Darebin - North
Whittlesea - Wallan
Banyule
Nillumbik - Kinglake
Boroondara
Whitehorse - West
Manningham - West
Manningham - East
Maroondah
Yarra Ranges
Stonnington - East
Monash
Whitehorse - East
Cardinia
Stonnington - West
Knox
Casey - North
Glen Eira
Dandenong
Bayside
Kingston
Frankston
Mornington Peninsula
Geelong
Surf Coast - Bellarine Peninsula
Warrnambool - Otway Ranges
Barwon - West
Glenelg - Southern Grampians
Limestone Coast
Grampians
Creswick - Daylesford - Ballan
Ballarat
Heathcote - Castlemaine - Kyneton
Maryborough - Pyrenees
Murray River - Swan Hill
Sunbury
Macedon Ranges
Upper Goulburn Valley
Loddon - Elmore
Mildura
Murray and Mallee
Campaspe
Bendigo
Shepparton
Moira
Wangaratta - Benalla
Wodonga - Alpine
Gippsland - East
Wellington
Casey - South
Baw Baw
Gippsland - South West
Latr

In [41]:
print(melbourne_venues.shape)
melbourne_venues.head()

(3521, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Melbourne City,-37.817403,144.956776,Virgin Active Health Club,-37.818806,144.955917,Gym / Fitness Center
1,Melbourne City,-37.817403,144.956776,Patricia Coffee Brewers,-37.814598,144.958350,Coffee Shop
2,Melbourne City,-37.817403,144.956776,Bonnie Coffee Brewers,-37.818153,144.957636,Coffee Shop
3,Melbourne City,-37.817403,144.956776,Brim CC,-37.817764,144.954732,Japanese Restaurant
4,Melbourne City,-37.817403,144.956776,Holey Moley,-37.815447,144.954906,Mini Golf


In [42]:
melbourne_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Ballarat,52,52,52,52,52,52
Banyule,100,100,100,100,100,100
Barwon - West,3,3,3,3,3,3
Baw Baw,5,5,5,5,5,5
Bayside,100,100,100,100,100,100
Bendigo,66,66,66,66,66,66
Boroondara,100,100,100,100,100,100
Brimbank,66,66,66,66,66,66
Brunswick - Coburg,100,100,100,100,100,100


In [43]:
print('There are {} unique categories.'.format(len(melbourne_venues['Venue Category'].unique())))

There are 251 unique categories.


# Step 3: Toronto neighbourhood analysis

In [44]:
# one hot encoding
melbourne_onehot = pd.get_dummies(melbourne_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
melbourne_onehot['Neighborhood'] = melbourne_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [melbourne_onehot.columns[-1]] + list(melbourne_onehot.columns[:-1])
melbourne_onehot = melbourne_onehot[fixed_columns]

melbourne_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,Airport,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Business Service,Café,Cambodian Restaurant,Campground,Cantonese Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Creperie,Cretan Restaurant,Cricket Ground,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Drive-in Theater,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,General Travel,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Health Food Store,History Museum,Hockey Field,Hookah Bar,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Multiplex,Museum,Music Store,Music Venue,National Park,Nature Preserve,Night Market,Noodle House,Opera House,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Outdoors & Recreation,Outlet Mall,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Racecourse,Racetrack,Record Shop,Recreation Center,Rental Car Location,Resort,Restaurant,River,Road,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Skating Rink,Ski Area,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Street Food Gathering,Supermarket,Surf Spot,Sushi Restaurant,Swim School,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Vineyard,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Winery,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Melbourne City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [45]:
# Group toronto neighbourhoods

melbourne_grouped = melbourne_onehot.groupby('Neighborhood').mean().reset_index()
melbourne_grouped

,Neighborhood,Afghan Restaurant,African Restaurant,Airport,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Business Service,Café,Cambodian Restaurant,Campground,Cantonese Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Creperie,Cretan Restaurant,Cricket Ground,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Drive-in Theater,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,General Travel,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Health Food Store,History Museum,Hockey Field,Hookah Bar,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Multiplex,Museum,Music Store,Music Venue,National Park,Nature Preserve,Night Market,Noodle House,Opera House,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Outdoors & Recreation,Outlet Mall,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Racecourse,Racetrack,Record Shop,Recreation Center,Rental Car Location,Resort,Restaurant,River,Road,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Skating Rink,Ski Area,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Street Food Gathering,Supermarket,Surf Spot,Sushi Restaurant,Swim School,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Vineyard,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Winery,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Ballarat,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.019231,0.000000,0.000000,0.019231,0.019231,0.000000,0.00,0.019231,0.076923,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.019231,0.00,0.000000,0.019231,0.019231,0.000

In [46]:
num_top_venues = 5

for hood in melbourne_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = melbourne_grouped[melbourne_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ballarat----
           venue  freq
0            Bar  0.08
1   Burger Joint  0.06
2  Grocery Store  0.06
3           Café  0.06
4     Restaurant  0.04


----Banyule----
                venue  freq
0                Café  0.26
1  Italian Restaurant  0.05
2                Park  0.05
3              Bakery  0.04
4                 Bar  0.04


----Barwon - West----
                        venue  freq
0  Construction & Landscaping  0.33
1                        Café  0.33
2           Convenience Store  0.33
3           Afghan Restaurant  0.00
4                 Pastry Shop  0.00


----Baw Baw----
                           venue  freq
0                    Supermarket   0.4
1                          Hotel   0.2
2               Tapas Restaurant   0.2
3                         Bakery   0.2
4  Paper / Office Supplies Store   0.0


----Bayside----
         venue  freq
0         Café  0.32
1        Beach  0.05
2          Bar  0.05
3  Coffee Shop  0.04
4       Bakery  0.03


----Bendigo----
     

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = melbourne_grouped['Neighborhood']

for ind in np.arange(melbourne_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(melbourne_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ballarat,Bar,Grocery Store,Burger Joint,Café,Hotel,History Museum,Gym,Fast Food Restaurant,Steakhouse,Coffee Shop
1,Banyule,Café,Park,Italian Restaurant,Bar,Pizza Place,Bakery,Indian Restaurant,Grocery Store,Japanese Restaurant,Lebanese Restaurant
2,Barwon - West,Convenience Store,Construction & Landscaping,Café,Filipino Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food & Drink Shop,Zoo Exhibit,Farmers Market
3,Baw Baw,Supermarket,Hotel,Bakery,Tapas Restaurant,Football Stadium,Food Truck,Food Service,Food Court,Farmers Market,French Restaurant
4,Bayside,Café,Beach,Bar,Coffee Shop,Sushi Restaurant,Bakery,Park,Restaurant,Bagel Shop,Liquor Store


# Step 4: Cluster neighbourhoods

In [54]:
# set number of clusters
kclusters = 5

melbourne_grouped_clustering = melbourne_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(melbourne_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 2, 0, 2, 0, 2, 2, 2, 2])

In [55]:
melbourne_merged.head()

,long,lat,neighbourhood
0,144.956776,-37.817403,Melbourne City
1,144.970161,-37.844246,Port Phillip
2,144.907953,-37.807101,Maribyrnong
3,144.880556,-37.838242,Hobsons Bay
4,144.829945,-37.777204,Brimbank


In [61]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

melbourne_merged = df_mel

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
melbourne_merged = melbourne_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='neighbourhood')

melbourne_merged.head()

,long,lat,neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,144.956776,-37.817403,Melbourne City,2.0,Café,Coffee Shop,Park,Bookstore,Hotel,Whisky Bar,Plaza,Gym / Fitness Center,Australian Restaurant,Bar
1,144.970161,-37.844246,Port Phillip,2.0,Café,Park,Coffee Shop,Ice Cream Shop,Pub,Beach,Theater,Dessert Shop,Performing Arts Venue,Gastropub
2,144.907953,-37.807101,Maribyrnong,2.0,Café,Coffee Shop,Bar,Bakery,Burger Joint,Park,Pub,Brewery,Gym,Vietnamese Restaurant
3,144.880556,-37.838242,Hobsons Bay,2.0,Café,Vietnamese Restaurant,Bar,Bakery,Indian Restaurant,Asian Restaurant,Sandwich Place,Pub,Beach,Park
4,144.829945,-37.777204,Brimbank,2.0,Café,Grocery Store,Vietnamese Restaurant,Indian Restaurant,Pizza Place,Gym,Electronics Store,Pub,Bakery,Chinese Restaurant


In [62]:
melbourne_merged = melbourne_merged.dropna(subset=['Cluster Labels'])

In [64]:
# Map the 5 melbourne neighbourhood clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(melbourne_merged['lat'], melbourne_merged['long'], melbourne_merged['neighbourhood'], melbourne_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Cluster 1

In [66]:
melbourne_merged.loc[melbourne_merged['Cluster Labels'] == 0, melbourne_merged.columns[[1] + list(range(5, melbourne_merged.shape[1]))]]

,lat,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,-37.782720,Grocery Store,Portuguese Restaurant,Café,Shopping Mall,Supermarket,Train Station,Racetrack,Beer Garden,Malay Restaurant
6,-37.750456,Train Station,Convenience Store,Café,Food & Drink Shop,Filipino Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Zoo Exhibit
15,-37.666070,Grocery Store,Gym,Convenience Store,Electronics Store,Supermarket,Gym / Fitness Center,Sandwich Place,Gas Station,Shopping Mall
17,-37.711787,Supermarket,Fast Food Restaurant,Gym / Fitness Center,Thai Restaurant,Theater,Coffee Shop,Electronics Store,Pub,Gym
22,-37.778595,Fast Food Restaurant,Supermarket,Japanese Restaurant,Shopping Mall,Coffee Shop,Sandwich Place,Grocery Store,Portuguese Restaurant,Mexican Restaurant
23,-37.737629,Supermarket,Convenience Store,Café,Portuguese Restaurant,Sandwich Place,Department Store,Burger Joint,Farm,Garden
29,-37.864951,Grocery Store,Malay Restaurant,Fast Food Restaurant,Sandwich Place,Convenience Store,Pub,Shopping Mall,Supermarket,Park
30,-37.917932,Grocery Store,Supermarket,Gas Station,Fast Food Restaurant,Shopping Mall,Portuguese Restaurant,Convenience Store,Nature Preserve,Theater
32,-37.945149,Café,Supermarket,Bakery,Grocery Store,Sandwich Place,Shopping Mall,Portuguese Restaurant,Burger Joint,Indonesian Restaurant
35,-38.072748,Beach,Café,Pizza Place,Fast Food Restaurant,Supermarket,Pub,Train Station,Brewery,Sandwich Place


# Examine Cluster 2

In [67]:
melbourne_merged.loc[melbourne_merged['Cluster Labels'] == 1, melbourne_merged.columns[[1] + list(range(5, melbourne_merged.shape[1]))]]

,lat,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,-37.025316,Pub,Zoo Exhibit,Farm,French Restaurant,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop


# Examine Cluster 3

In [68]:
melbourne_merged.loc[melbourne_merged['Cluster Labels'] == 2, melbourne_merged.columns[[1] + list(range(5, melbourne_merged.shape[1]))]]

,lat,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,-37.817403,Coffee Shop,Park,Bookstore,Hotel,Whisky Bar,Plaza,Gym / Fitness Center,Australian Restaurant,Bar
1,-37.844246,Park,Coffee Shop,Ice Cream Shop,Pub,Beach,Theater,Dessert Shop,Performing Arts Venue,Gastropub
2,-37.807101,Coffee Shop,Bar,Bakery,Burger Joint,Park,Pub,Brewery,Gym,Vietnamese Restaurant
3,-37.838242,Vietnamese Restaurant,Bar,Bakery,Indian Restaurant,Asian Restaurant,Sandwich Place,Pub,Beach,Park
4,-37.777204,Grocery Store,Vietnamese Restaurant,Indian Restaurant,Pizza Place,Gym,Electronics Store,Pub,Bakery,Chinese Restaurant
7,-37.761235,Pub,Electronics Store,Coffee Shop,Bakery,Gym,Vietnamese Restaurant,Grocery Store,Shopping Mall,Indian Restaurant
8,-37.766184,Bar,Bakery,Coffee Shop,Middle Eastern Restaurant,Park,Farmers Market,Thai Restaurant,Department Store,Vietnamese Restaurant
9,-37.726746,Supermarket,Grocery Store,Sandwich Place,Portuguese Restaurant,Rental Car Location,Park,Liquor Store,Breakfast Spot,Shopping Mall
10,-37.691916,Airport Lounge,Coffee Shop,Middle Eastern Restaurant,Fast Food Restaurant,Rental Car Location,Supermarket,Juice Bar,Basketball Court,Electronics Store
11,-37.786971,Coffee Shop,Bakery,Bar,Park,Vegetarian / Vegan Restaurant,Italian Restaurant,Wine Bar,Cocktail Bar,Bookstore


# Examine Cluster 4

In [69]:
melbourne_merged.loc[melbourne_merged['Cluster Labels'] == 3, melbourne_merged.columns[[1] + list(range(5, melbourne_merged.shape[1]))]]

,lat,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,-36.894465,Zoo Exhibit,Farmers Market,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop


# Examine Cluster 5

In [70]:
melbourne_merged.loc[melbourne_merged['Cluster Labels'] == 4, melbourne_merged.columns[[1] + list(range(5, melbourne_merged.shape[1]))]]

,lat,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,-37.448455,Spa,Locksmith,Zoo Exhibit,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck,Food Service,Food Court
